In [2]:
import matplotlib as mpl
import keras.backend as K
from keras.layers import Dropout, Reshape, Flatten, LeakyReLU, Activation, Dense, BatchNormalization, SpatialDropout2D
from keras.layers.convolutional import Conv2D, SeparableConv2D, UpSampling2D, MaxPooling2D, AveragePooling2D
from keras.regularizers import L1L2
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.datasets import cifar10
from libs.keras_adversarial.image_grid_callback import ImageGridCallback
from libs.keras_adversarial import AdversarialModel, simple_gan, gan_targets, fix_names
from libs.keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling, AdversarialOptimizerAlternating
from libs.utils.image_utils import dim_ordering_unfix, dim_ordering_shape
import numpy as np
import pandas as pd

In [3]:
# This line allows mpl to run with no DISPLAY defined
mpl.use("Agg")

In [4]:
def model_generator():
    model = Sequential()
    #nch = 256
    nch = 512
    reg = lambda: L1L2(l1=1e-7, l2=1e-7)
    #h = 5
    h = 7
    model.add(Dense(nch * 4 * 4, input_dim=100, kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(Reshape(dim_ordering_shape((nch, 4, 4))))
    model.add(Conv2D(int(nch / 2), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(int(nch / 2), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(int(nch / 4), (h, h), padding="same", kernel_regularizer=reg()))
    model.add(BatchNormalization())
    model.add(LeakyReLU(0.2))
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(3, (h, h), padding="same", kernel_regularizer=reg()))
    model.add(Activation("sigmoid"))
    
    return model

In [5]:
def model_discriminator():
    #nch = 256
    nch = 512
    #h = 5
    h = 7
    reg = lambda: L1L2(l1=1e-7, l2=1e-7)

    c1 = SeparableConv2D(int(nch / 4),
                (h, h),
                padding="same",
                kernel_regularizer=reg(),
                input_shape=dim_ordering_shape((3, 32, 32)))
    c2 = SeparableConv2D(int(nch / 2),
                (h, h),
                padding="same",
                kernel_regularizer=reg())
    c3 = SeparableConv2D(nch,
                (h, h),
                padding="same",
                kernel_regularizer=reg())
    c4 = SeparableConv2D(1,
                (h, h),
                padding="same",
                kernel_regularizer=reg())

    def m(dropout):
        model = Sequential()
        model.add(c1)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c2)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c3)
        model.add(SpatialDropout2D(dropout))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(LeakyReLU(0.2))
        model.add(c4)
        model.add(AveragePooling2D(pool_size=(4, 4), padding="valid"))
        model.add(Flatten())
        model.add(Activation('sigmoid'))
        
        return model
    return m

In [6]:
from keras.preprocessing.image import img_to_array, load_img
import os
import glob
from sklearn.model_selection import train_test_split

data_dirname = os.path.join('dataset', 'predicted_auto2', '*')

def load_data():
    original_data = []
    for file in glob.glob(data_dirname):
        image = load_img(file).resize((32, 32))
        array = img_to_array(image)
        original_data.append(array)
    data = np.array(original_data) / 255
    print(f'original_data shape: {data.shape}')
    (xtrain, xtest, ytrain, ytest) = train_test_split(data, data, test_size=0.1)
    print(f'xtrain shape: {xtrain.shape}, xtest shape: {xtest.shape}')
    
    return (
        xtrain,
        xtest
    )

In [7]:
class CustomImageGridCallback(ImageGridCallback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 100 != 0:
            return
        super().on_epoch_end(epoch, logs)

In [8]:
def example_gan(adversarial_optimizer, path, opt_g, opt_d, nb_epoch,
                generator, discriminator, latent_dim,
                targets=gan_targets, loss="binary_crossentropy"):
    csvpath = os.path.join(path, "history.csv")
    if os.path.exists(csvpath):
        print("Already exists: {}".format(csvpath))
        return

    print("Training: {}".format(csvpath))
    # gan (x - > yfake, yreal), z is gaussian generated on GPU
    # can also experiment with uniform_latent_sampling
    d_g = discriminator(0)
    d_d = discriminator(0.5)
    generator.summary()
    d_d.summary()
    gan_g = simple_gan(generator, d_g, None)
    gan_d = simple_gan(generator, d_d, None)
    x = gan_g.inputs[1]
    z = normal_latent_sampling((latent_dim,))(x)
    # estiminate z from inputs
    gan_g = Model([x], fix_names(gan_g([z, x]), gan_g.output_names))
    gan_d = Model([x], fix_names(gan_d([z, x]), gan_d.output_names))

    # build adversarial model
    model = AdversarialModel(player_models=[gan_g, gan_d],
                             player_params=[generator.trainable_weights,
                                            d_d.trainable_weights],
                             player_names=["generator", "discriminator"])
    model.adversarial_compile(adversarial_optimizer=adversarial_optimizer,
                              player_optimizers=[opt_g, opt_d],
                              loss=loss)

    # create callback to generate images
    zsamples = np.random.normal(size=(5 * 5, latent_dim))

    def generator_sampler():
        xpred = generator.predict(zsamples)
        xpred = dim_ordering_unfix(xpred.transpose((0, 2, 3, 1)))
        return xpred.reshape((5, 5) + xpred.shape[1:])

    generator_cb = CustomImageGridCallback(
                    os.path.join(path, "epoch-{:03d}.png"),
                    generator_sampler, cmap=None)

    callbacks = [generator_cb]
    if K.backend() == "tensorflow":
        callbacks.append(
            TensorBoard(log_dir=os.path.join(path, "logs"),
                        histogram_freq=0, write_graph=True, write_images=True))

    # train model
    xtrain, xtest = load_data()
    y = targets(xtrain.shape[0])
    ytest = targets(xtest.shape[0])
    history = model.fit(x=xtrain, y=y, validation_data=(xtest, ytest),
                        callbacks=callbacks, epochs=nb_epoch,
                        batch_size=32)

    # save history to CSV
    df = pd.DataFrame(history.history)
    df.to_csv(csvpath)

    # save models
    generator.save(os.path.join(path, "generator.h5"))
    d_d.save(os.path.join(path, "discriminator.h5"))

In [9]:
import os

def main():
    # z \in R^100
    latent_dim = 100
    # x \in R^{28x28}
    # generator (z -> x)
    generator = model_generator()
    # discriminator (x -> y)
    discriminator = model_discriminator()
    if not os.path.exists("generated/pikachu"):
        os.mkdir("generated/pikachu")
    example_gan(AdversarialOptimizerSimultaneous(), "generated/pikachu",
                opt_g=Adam(1e-4, decay=1e-5),
                opt_d=Adam(1e-3, decay=1e-5),
                nb_epoch=50001, generator=generator, discriminator=discriminator,
                latent_dim=latent_dim)


if __name__ == "__main__":
    main()




Instructions for updating:
keep_dims is deprecated, use keepdims instead

Training: generated/pikachu\history.csv


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
batch_normalization_1 (Batch (None, 8192)              32768     
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 256)         6422784   
_________________________________________________________________
batch_normalization_2 (Batch (Non

In [3]:
import tensorflow

print(tensorflow.__version__)

ModuleNotFoundError: No module named 'tensorflow'